In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
df = pd.read_json('../app/files/data.json')

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14337 entries, 0 to 14336
Data columns (total 44 columns):
acct_type             14337 non-null object
approx_payout_date    14337 non-null int64
body_length           14337 non-null int64
channels              14337 non-null int64
country               14256 non-null object
currency              14337 non-null object
delivery_method       14321 non-null float64
description           14337 non-null object
email_domain          14337 non-null object
event_created         14337 non-null int64
event_end             14337 non-null int64
event_published       14238 non-null float64
event_start           14337 non-null int64
fb_published          14337 non-null int64
gts                   14337 non-null float64
has_analytics         14337 non-null int64
has_header            8928 non-null float64
has_logo              14337 non-null int64
listed                14337 non-null object
name                  14337 non-null object
name_length      

In [23]:
df.head()

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,...,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,fraudster_event,1266062400,3852,5,US,USD,0.0,"<p><a href=""http://s432.photobucket.com/albums...",gmail.com,1262739706,...,"[{'event_id': 527017, 'cost': 25.0, 'availabil...",36,1259613950,1,717 Washington Avenue,US,25.777471,-80.133433,INK Nightclub - South Beach,FL
1,premium,1296720000,3499,0,US,USD,1.0,"<p>Join us for a quick, one-night, community-b...",ruf.org,1293832670,...,"[{'event_id': 786878, 'cost': 35.0, 'availabil...",149,1280942776,3,,US,32.776566,-79.930922,"The Charleston, SC area",SC
2,premium,1296172800,2601,8,US,USD,1.0,"<h3><span class=""subcategory""><strong>Teacher ...",pvsd.k12.ca.us,1291090956,...,"[{'event_id': 787337, 'cost': 93.51, 'availabi...",214,1272559388,3,10100 Pioneer Blvd Suite 100,US,33.944201,-118.080419,Los Angeles County Office of Education,CA
3,premium,1388966400,12347,6,IE,EUR,1.0,"<p style=""margin-bottom: 1.3em; padding-bottom...",irishtabletennis.com,1360681570,...,"[{'event_id': 885645, 'cost': 25.0, 'availabil...",889,1283870102,3,,None,NaN,NaN,None,None
4,premium,1297900800,2417,11,US,USD,0.0,<p>Writers and filmmakers need to understand t...,artsandbusinesscouncil.org,1291994666,...,"[{'event_id': 1114349, 'cost': 150.0, 'availab...",35,1288984065,3,One Marina Park Drive,US,42.353848,-71.044276,Fish & Richardson,MA


What is a fraud event?

In [24]:
df['acct_type'].unique().tolist()

['fraudster_event',
 'premium',
 'spammer_warn',
 'fraudster',
 'spammer_limited',
 'spammer_noinvite',
 'locked',
 'tos_lock',
 'tos_warn',
 'fraudster_att',
 'spammer_web',
 'spammer']

Breakout of `acct_types`

In [25]:
df.groupby('acct_type')['approx_payout_date'].count().reset_index().sort_values('approx_payout_date', ascending = False)

,acct_type,approx_payout_date
4,premium,12373
2,fraudster_event,851
0,fraudster,437
6,spammer_limited,218
8,spammer_warn,144
11,tos_warn,91
7,spammer_noinvite,84
10,tos_lock,77
3,locked,54
1,fraudster_att,5


## What do frauds look like

In [26]:
frauds = ['fraudster_event', 'fraudster', 'fraudster_att']
df['frauds'] = [True if i in frauds else False for i in df['acct_type']]
# df['frauds'].head()

In [27]:
df[df['frauds']].count()

acct_type             1293
approx_payout_date    1293
body_length           1293
channels              1293
country               1293
currency              1293
delivery_method       1281
description           1293
email_domain          1293
event_created         1293
event_end             1293
event_published       1199
event_start           1293
fb_published          1293
gts                   1293
has_analytics         1293
has_header             519
has_logo              1293
listed                1293
name                  1293
name_length           1293
num_order             1293
num_payouts           1293
object_id             1293
org_desc              1293
org_facebook          1238
org_name              1293
org_twitter           1238
payee_name            1293
payout_type           1293
previous_payouts      1293
sale_duration         1175
sale_duration2        1293
show_map              1293
ticket_types          1293
user_age              1293
user_created          1293
u

### EDA fraud % related to country

In [28]:
df_frauds = df[df['frauds']]
df_not_frauds = df[df['frauds']==False]

In [29]:
df_not_frauds.shape

(13044, 45)

In [30]:
df_f = df_frauds.groupby('country')['frauds'].count().reset_index().sort_values('frauds', ascending = False)

In [31]:
df_nf = df_not_frauds.groupby('country')['frauds'].count().reset_index().sort_values('frauds', ascending = False)

In [32]:
df_nf.columns = ['country','not fraud']

In [33]:
df_fraud_count = pd.merge(df_f, df_nf, on='country', how='outer')

In [34]:
df_fraud_count = df_fraud_count.fillna(value=0)

In [35]:
df_fraud_count['fraud_%'] = df_fraud_count['frauds']/(df_fraud_count['not fraud']+df_fraud_count['frauds'])

In [36]:
type(df_fraud_count)

pandas.core.frame.DataFrame

In [37]:
df_fraud_count = df_fraud_count.sort_values(['fraud_%'],ascending=False)

In [38]:
df_fraud_count = pd.get_dummies(df_fraud_count, columns = ['country'])

In [56]:
df_fraud_count.head()
#df_fraud_count = df_fraud_count.drop('country', axis=1)

,not fraud,fraud_%,country_,country_A1,country_AE,country_AR,country_AT,country_AU,country_BB,country_BE,...,country_SI,country_TH,country_TJ,country_TR,country_US,country_UY,country_VE,country_VI,country_VN,country_ZA
34,0.0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0.0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28,0.0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,0.0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26,0.0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_fraud_count.head()

### RF model attempt

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [40]:
y = df_fraud_count.pop('frauds').values
X = df_fraud_count.drop('fraud_%',axis =1)
X = X.drop('not fraud',axis =1)

In [57]:
X.head()

,country_,country_A1,country_AE,country_AR,country_AT,country_AU,country_BB,country_BE,country_BG,country_BS,...,country_SI,country_TH,country_TJ,country_TR,country_US,country_UY,country_VE,country_VI,country_VN,country_ZA
34,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [51]:
rf = RandomForestClassifier(n_estimators=100)

In [52]:
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [59]:
rf.score(X_test,y_test)

0.44444444444444442

In [54]:
len(rf.predict(X_test))

18

### Time conversion function

In [ ]:
time_cols = ['approx_payout_date', 'event_created', 'event_published','event_start', 'user_created']

In [ ]:
for col in time_cols:
    df[col] = df[col].astype('datetime64[s]')

check

In [ ]:
print(df['approx_payout_date'].min(),
df['approx_payout_date'].max())

In [ ]:
df.listed.head()

### Cleaners

In [ ]:
df['num_previous_payouts'] = [len(i) for i in df['previous_payouts']]

In [ ]:
df['listed'] = [True if i=='y' else False for i in df['listed']]

### Currency vs Country

In [ ]:
df[['country','currency']].head()

In [ ]:
df.currency.unique()

### Interesting fields

In [ ]:
for_hist = ['body_length', 
            'channels',
            'country',
            'currency',
            'delivery_method',
            'email_domain',
            'fb_published',
            'has_analytics',
            'listed',
            'num_payouts', 
            'payout_type',
            'sale_duration', 
            'sale_duration2',
            'ticket_types',
            'user_age']

In [ ]:
df_clean = df[df['frauds'] == False]
df_dirty = df[df['frauds']]

In [ ]:
# fig, ax = plt.subplots(2)
df_clean['body_length'].hist(range = (0,100))

In [ ]:
df_dirty['body_length'].hist(range = (0,100))

### Dummy Variables for Countries

In [ ]:
df.head()

In [ ]:
df = pd.get_dummies(df, columns = ['country'])

In [ ]:
df['frauds'].head()

In [ ]:
df.groupby(["", "Size"]).size().reset_index(name="Time")

In [ ]:
df.head()

In [ ]:
#def create_dummy(column_name):
    #df = pd.get_dummies(df, columns = ['column_name'])

In [ ]:
df_country_dummy